## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
0,0,Yellowknife,62.4560,-114.3525,59.56,58,40,4.61,scattered clouds,CA,2022-06-20 13:57:35
1,1,Souillac,-20.5167,57.5167,70.18,88,75,14.97,light rain,MU,2022-06-20 14:01:40
2,2,Ushuaia,-54.8000,-68.3000,27.73,95,100,5.32,light snow,AR,2022-06-20 14:01:42
3,3,Mataura,-46.1927,168.8643,33.48,98,21,2.42,few clouds,NZ,2022-06-20 14:01:50
4,4,Wynyard,-41.0000,145.7167,53.98,92,100,14.43,moderate rain,AU,2022-06-20 13:57:19


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
1,1,Souillac,-20.5167,57.5167,70.18,88,75,14.97,light rain,MU,2022-06-20 14:01:40
5,5,Praya,14.9215,-23.5087,81.14,78,20,13.80,few clouds,CV,2022-06-20 13:58:59
7,7,Ugoofaaru,5.6667,73.0000,82.81,76,100,15.61,overcast clouds,MV,2022-06-20 14:02:00
11,11,Samana,19.2056,-69.3369,82.62,78,20,0.00,few clouds,DO,2022-06-20 14:02:17
13,13,Tarakan,3.3000,117.6333,82.72,71,13,7.20,few clouds,ID,2022-06-20 14:02:25


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        333
City           333
Lat            333
Lng            333
Max Temp       333
Humidity       333
Cloudiness     333
Wind Speed     333
Description    333
Country        332
Date           333
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df['country'] = preferred_cities_df['Country'].fillna(0)
cleaned_df = preferred_cities_df.loc[preferred_cities_df['Country'] !=0]
cleaned_df.count()

/var/folders/8m/3l3klsyx5373b_s8jmcpgp_80000gn/T/ipykernel_14332/3282142463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preferred_cities_df['country'] = preferred_cities_df['Country'].fillna(0)


City_ID        333
City           333
Lat            333
Lng            333
Max Temp       333
Humidity       333
Cloudiness     333
Wind Speed     333
Description    333
Country        332
Date           333
country        333
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaned_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Souillac,MU,70.18,light rain,-20.5167,57.5167,
5,Praya,CV,81.14,few clouds,14.9215,-23.5087,
7,Ugoofaaru,MV,82.81,overcast clouds,5.6667,73.0000,
11,Samana,DO,82.62,few clouds,19.2056,-69.3369,
13,Tarakan,ID,82.72,few clouds,3.3000,117.6333,
14,Ribeira Grande,PT,70.20,few clouds,38.5167,-28.7000,
16,Omboue,GA,78.57,broken clouds,-1.5746,9.2618,
19,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,
21,The Valley,AI,85.10,few clouds,18.2170,-63.0578,
22,Atuona,PF,77.61,light rain,-9.8000,-139.0333,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f'{lat},{lng}'

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].fillna(0)
hotel_final_df = hotel_df.loc[hotel_df['Hotel Name'] != 0]
hotel_final_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Souillac,MU,70.18,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
5,Praya,CV,81.14,few clouds,14.9215,-23.5087,LT Aparthotel
7,Ugoofaaru,MV,82.81,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
11,Samana,DO,82.62,few clouds,19.2056,-69.3369,The Bannister - Hotel & Yacht Club
13,Tarakan,ID,82.72,few clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan


In [20]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>\
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_final_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_final_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[['Lat', 'Lng']]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))